In [ ]:
import numpy as np 
import pandas as pd

import cv2

from fastai.vision import *
from fastai.metrics import error_rate

### Visualizing Some Images

In [ ]:
train_dataset_path = "/kaggle/input/blood-cells/dataset2-master/dataset2-master/images/TEST" 

CATEGORIES    = ['EOSINOPHIL','LYMPHOCYTE','MONOCYTE','NEUTROPHIL']

for k in range(3):
    i=0
    plt.figure(figsize=(25,15))
    for category in CATEGORIES:
        plt.subplot(5, 5, i+1)
        plt.yticks([])
        plt.xticks([])
        path=train_dataset_path + '/' + category
        image_p=os.listdir(path)
        plt.title(category , color='tomato').set_size(15)
        plt.axis('off')
        image = cv2.imread(os.path.join(path, image_p[k])) 
        image = image[:, :, [2, 1, 0]] 
        plt.imshow(image)
        i+=1


### Path of the All Datasets

In [ ]:
path = Path('/kaggle/input/blood-cells/dataset2-master/dataset2-master/images')
path.ls()

### Seperating Dataset

In [ ]:
data = ImageDataBunch.from_folder(path, train='TRAIN', test='TEST', valid_pct=0.20,
        ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)

In [ ]:
data.classes

In [ ]:
print("Total number of classes are :", data.c) 
print("Total train dataset is :", len(data.train_ds)) 
print("Total valid dataset is :", len(data.valid_ds))

### Model

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [ ]:
learn.fit_one_cycle(2)

### Directory and Plot

In [ ]:
learn.unfreeze()

In [ ]:
learn.model_dir=Path('/kaggle/working_dir')
learn.lr_find()

In [ ]:
learn.recorder.plot()

### Fit Model 3 times

In [ ]:
learn.fit_one_cycle(3, max_lr=slice(5e-5,5e-4))

In [ ]:
learn.save('model')

### Classification Report

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()